<a href="https://colab.research.google.com/github/Ayman850/COMP257/blob/main/COMP257_Ass2_AymanYasin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment2 - Question 1

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, silhouette_score
from sklearn.cluster import KMeans, DBSCAN

## 1. Retrieve and load Olivetti faces

In [ ]:
# Retrieve the Olivetti faces

olivetti_faces = fetch_olivetti_faces()

In [ ]:
#shape of the data
X = olivetti_faces.data
y = olivetti_faces.target
images = olivetti_faces.images # 64x64 version of .data
print(X.shape)
print(y.shape)

In [ ]:
# I Plot first 15 images
fig, axes = plt.subplots(3, 5, figsize=(10, 5))

for i, ax in enumerate(axes.ravel()):
    ax.imshow(images[i], cmap='gray')
    ax.set_title(f"ID: {y[i]}")
    ax.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# From Reference [1]
# I use the similar notebook which exploring this dataset and using the same function aswell

def show_40_distinct_people(images, unique_ids):
    #Creating 4X10 subplots in  18x9 figure size
    fig, axarr=plt.subplots(nrows=4, ncols=10, figsize=(18, 9))
    #For easy iteration flattened 4X10 subplots matrix to 40 array
    axarr=axarr.flatten()

    #iterating over user ids
    for unique_id in unique_ids:
        image_index=unique_id*10
        axarr[unique_id].imshow(images[image_index], cmap='gray')
        axarr[unique_id].set_xticks([])
        axarr[unique_id].set_yticks([])
        axarr[unique_id].set_title("face id:{}".format(unique_id))
    plt.suptitle("There are 40 distinct people in the dataset")

show_40_distinct_people(images, np.unique(y))

#here now it ploting 1 image per face id, now we can see 40 diffrent faces, totall is 400, 10 images for 40 people

## 2. Split train, val, test with stratified sampling

In [ ]:
# I choose because for 1 subject there are 10 images

# train has 8 images for 1 person, 1 images for validation, and 1 images for test
'''
Split is:
80% train
10% validation
10% test
to ensure we have a lot of data to train,
and also still have some left for validation and testing
'''
# First split into train + validation (90%) and test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y)

# Now split the train + validation set (90%) into train (80%) and validation (10%)
# 10/90 = 0.1111
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42, stratify=y_train_val)  # I using stratify to make it balance

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
#Here is just to check what I said is correct
# From Reference [1]
y_frame=pd.DataFrame()
y_frame['subject ids']=y_train
y_frame.groupby(['subject ids']).size().plot.bar(figsize=(15,8),title="Number of Samples for Each Classes (TRAIN)")

In [ ]:
# From Reference [1]
y_frame=pd.DataFrame()
y_frame['subject ids']=y_val
y_frame.groupby(['subject ids']).size().plot.bar(figsize=(15,8),title="Number of Samples for Each Classes (VALIDATION)")

In [ ]:
# From Reference [1]
y_frame=pd.DataFrame()
y_frame['subject ids']=y_test
y_frame.groupby(['subject ids']).size().plot.bar(figsize=(15,8),title="Number of Samples for Each Classes (TEST)")

## 3. Use k-fold CV, train a classifier, and evaluate on validation set

In [ ]:
# Use SVC with linear kernel
clf = SVC(kernel='linear', random_state=42)

# Instantiate StratifiedKFold
# use n_splits=4 so the 8 training samples per class can be evenly distributed of 2 samples each
kf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

# Run k-fold CV
cv_scores = cross_val_score(clf, X_train, y_train, cv=kf, scoring='accuracy')

# Print scores
print(f"Cross-validation accuracy scores: {cv_scores}")
print(f"Mean cross-validation accuracy: {cv_scores.mean():.4f}")

# Train the classifier on the entire training set
clf.fit(X_train, y_train)

# Evaluate the classifier on the validation set
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation set accuracy: {val_accuracy:.4f}")

## 4. Use K-Means to reduce the dimensionality of the set. Provide your rationale for the similarity measure used to perform the clustering. Use the silhouette score approach to choose the number of clusters.

In [ ]:
# Try different values of k from 20 to 150 bcz of 40 faces, where 20 is lower then 40 and 150 is high (reson the rest I mentioned on report)
cluster_range = range(20, 150)
silhouette_scores = []

# Perform KMeans clustering and compute the silhouette score for each number of clusters
for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, cluster_labels)
    silhouette_scores.append(silhouette_avg)
    print(f"For k = {k}, the silhouette score is {silhouette_avg:.4f}")

# Plot silhouette scores to visualize the optimal number of clusters
plt.figure(figsize=(8, 5))
plt.plot(cluster_range, silhouette_scores, marker='o')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.grid(True)
plt.show()

# Choose the number of clusters with the highest silhouette score
optimal_k = cluster_range[np.argmax(silhouette_scores)]
print(f"The optimal number of clusters is: {optimal_k}")


In [ ]:
# here I used optimal_k to reduce
optimal_k = 129
# Reduce dimensionality to the optimal k found previously
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42)
X_reduced = kmeans_optimal.fit_transform(X)

In [ ]:
# here you can see the result for reduce
print(X.shape)
print(X_reduced.shape)

In [ ]:
# labels will become which cluster the data belongs to
# instead of the person id
y_reduced = kmeans_optimal.labels_
print(y_reduced.shape)
print(y_reduced)

In [ ]:
y_frame=pd.DataFrame()
y_frame['subject ids']=y_reduced
y_frame.groupby(['subject ids']).size().plot.bar(figsize=(15,8),title="Number of Samples for Each Classes")

## 5. Use set from step 4 to train classifier as in step 3

In [ ]:
X = X_reduced
y = y_reduced

print(X.shape)
print(y.shape)

In [ ]:
# First split into train + validation (90%) and test (15%)
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42)

# Now split the train + validation set (90%) into train (80%) and validation (10%)
# 10/90 = 0.1111
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.1111, random_state=42)

In [ ]:
# Use SVC with linear kernel
clf = SVC(kernel='linear', random_state=42)

# Instantiate StratifiedKFold
kf = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Run k-fold CV
cv_scores = cross_val_score(clf, X_train, y_train, cv=kf, scoring='accuracy')

# Print scores
print(f"Cross-validation accuracy scores: {cv_scores}")
print(f"Mean cross-validation accuracy: {cv_scores.mean():.4f}")

# Train the classifier on the entire training set
clf.fit(X_train, y_train)

# Evaluate the classifier on the validation set
y_val_pred = clf.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)

print(f"Validation set accuracy: {val_accuracy:.4f}")

## 6. Apply DBSCAN to dataset. Preprocess images and convert into feature vectors. Use DBSCAN to group images based on density.

### a. Using K-Means like previous step

In [ ]:
# Reload data
olivetti_faces = fetch_olivetti_faces()
X = olivetti_faces.data
y = olivetti_faces.target

optimal_k = 125
# Reduce dimensionality to the optimal k found previously
kmeans_optimal = KMeans(n_clusters=optimal_k, random_state=42)
X_reduced = kmeans_optimal.fit_transform(X)
print(f"Before: {X.shape}")
print(f"After: {X_reduced.shape}")

In [ ]:
# use DBSCAN
eps = 10
min_samples = 4
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
dbscan.fit(X_reduced)

labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Subtract 1 for noise (-1 label)
cluster_counts = Counter(labels)
n_noise = list(labels).count(-1)

print(f'eps: {eps}')
print(f'min_samples: {min_samples}')
print(f'Number of clusters: {n_clusters}')
print(f'Number of noise points: {n_noise}')
print(f"Number of samples in each cluster:")
for label, count in sorted(cluster_counts.items()):
    if label == -1:
        print(f"Noise (label -1): {count} samples")
    else:
        print(f"Cluster {label}: {count} samples")
if n_clusters > 1:
    silhouette_avg = silhouette_score(X_reduced, labels)
    print(f"Silhouette Score: {silhouette_avg}")

### b. Using PCA to reduce to 2D

In [ ]:
# Reload data
olivetti_faces = fetch_olivetti_faces()
X = olivetti_faces.data
y = olivetti_faces.target

# Reduce dimensionality to 2D with PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print(f"Before: {X.shape}")
print(f"After: {X_reduced.shape}")

In [ ]:
# use DBSCAN
eps = 1.5
min_samples = 4
dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')
dbscan.fit(X_reduced)

# From Reference [3]
# Define function to plot DBSCAN results
def plot_dbscan(dbscan, X, size, show_xlabels=True, show_ylabels=True):
    core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
    core_mask[dbscan.core_sample_indices_] = True
    anomalies_mask = dbscan.labels_ == -1
    non_core_mask = ~(core_mask | anomalies_mask)

    cores = dbscan.components_
    anomalies = X[anomalies_mask]
    non_cores = X[non_core_mask]

    plt.scatter(cores[:, 0], cores[:, 1], c=dbscan.labels_[core_mask], marker='+', s=size, cmap="Paired")
    plt.scatter(anomalies[:, 0], anomalies[:, 1], c="r", marker="x", s=100)
    plt.scatter(non_cores[:, 0], non_cores[:, 1], c=dbscan.labels_[non_core_mask], marker="*")

    if show_xlabels:
        plt.xlabel("$x_1$")
    if show_ylabels:
        plt.ylabel("$x_2$", rotation=0)
    plt.title(f"eps={dbscan.eps:.2f}, min_samples={dbscan.min_samples}")
    plt.grid()
    plt.gca().set_axisbelow(True)

# Plotting the results for DBSCAN with eps=0.05
plt.figure(figsize=(6, 4))
plot_dbscan(dbscan, X_reduced, size=100)
plt.show()

labels = dbscan.labels_
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Subtract 1 for noise (-1 label)
cluster_counts = Counter(labels)
n_noise = list(labels).count(-1)

print(f'eps: {eps}')
print(f'min_samples: {min_samples}')
print(f'Number of clusters: {n_clusters}')
print(f'Number of noise points: {n_noise}')
print(f"Number of samples in each cluster:")
for label, count in sorted(cluster_counts.items()):
    if label == -1:
        print(f"Noise (label -1): {count} samples")
    else:
        print(f"Cluster {label}: {count} samples")
if n_clusters > 1:
    silhouette_avg = silhouette_score(X_reduced, labels)
    print(f"Silhouette Score: {silhouette_avg}")

## References  
[1] https://www.kaggle.com/code/serkanpeldek/face-recognition-on-olivetti-dataset  
[2] https://e.centennialcollege.ca/d2l/le/content/1186996/viewContent/14778766/View (K-Means lab)  
[3] https://e.centennialcollege.ca/d2l/le/content/1186996/viewContent/14778765/View (DBSCAN lab)